# 1. 
## 기본적인 세팅을 하는 부분입니다:
기본 라이브러리 import와 GPU 사용이 가능한지 확인하고 난수를 설정해줍니다.</br>
이후 학습률이나 배치 사이즈 등의 기본 파라미터들의 값을 선언해줍니다.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms
 
device = 'cuda' if torch.cuda.is_available() else 'cpu' #torch.cuda.is_available(): cuda 사용가능하면 true를, 사용가능하지 않으면 false를 output으로 내놓습니다.
torch.manual_seed(777) #torch.manual_seed(777): pythorch에서 randomness를 제어하기 위하여 난수를 설정해줍니다.
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
print(device + " is available")
 
learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5

# 2.
## 딥러닝에 필요한 데이터를 불러오는 단계입니다:
여러 옵션들을 사용하여 데이터를 불러오고, 불러온 데이터들을 분석에 맞게 reshape 해줍니다.

In [ ]:
train_set = torchvision.datasets.MNIST( 
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor() 
    ])
)
test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)
 
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size) #DataLoader: data set에 대하여 반복가능한 함수입니다. train set을 batch size에 선언한 사이즈 만큼 잘라줍니다.
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)
 
examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

# 3.
## 모델을 생성하는 단계입니다: 
모델에 사용될 요소들을 정의 및 초기화 하고, 정의된 요소들을 잘 연결 하여 모델을 구성합니다.

In [ ]:
class ConvNet(nn.Module):
  def __init__(self): 
        super(ConvNet, self).__init__()

        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) #nn.Conv2d: in_channel:1, channel:2, kernal_size:5, 디폴트로 padding:0, dilation:1, group:1, bias:True 값을 가진 합성곱 신경망 입니다.
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) 
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) #nn.Dropout2d: p(요소가 기본적으로 0이 될 확률): 0.25, inplace(제자리에서 작업을 수행할지의 여부): True인 값을 랜덤하게 0으로 바꿔주는 dropout 함수입니다.
        self.mp = nn.MaxPool2d(2) #nn.MaxPool2d: kernel size가 2인 맥스풀링 함수입니다. 2*2에서 제일 큰수를 뽑아줍니다.
        self.fc1 = nn.Linear(320,100) 
        self.fc2 = nn.Linear(100,10) 

  def forward(self, x):
        x = F.relu(self.mp(self.conv1(x))) #conv2D에 maxpooling을 취한 뒤 해당 layer에 relu activation function을 적용합니다. relu는 음수에서는 0, 양수에서는 y=x의 값을 갖는 활성화 함수입니다.
        x = F.relu(self.mp(self.conv2(x))) 
        x = self.drop2D(x)
        x = x.view(x.size(0), -1) 
        x = self.fc1(x) 
        x = self.fc2(x) 
        return F.log_softmax(x) 

# 4.
## 학습 전 세팅하는 단계입니다:
사용할 모델과, 손실함수, 최적화 알고리즘을 변수를 통하여 선언해줍니다.

In [ ]:
model = ConvNet().to(device) 
criterion = nn.CrossEntropyLoss().to(device) #CrossEntropyLoss(): 손실함수로 crossentropy함수를 지정해줍니다. 크로스 엔트로피 loss fuction은 다중 분류를 위한 대표적인 손실함수 입니다.
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) # torch.optim.Adam(): 최적화 알고리즘은 아담 알고리즘으로 설정해줍니다. adam은 파라미터마다 다른 크기의 업데이트를 적용 해줍니다.

# 5.
## 모델 학습 단계입니다.:
위에서 설정한 epoch수 만큼 가중치를 업데이트 하면서 학습을 반복합니다
가중치는 미분과 경사하강법을 통하여 구합니다.

In [ ]:
for epoch in range(epochs): 
    avg_cost = 0

    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad() #zero_grad(): 모든 model의 gradient 값을 0으로 설정 - 초기화 이유 : pytorch에서는 gradient값들을 추후에 backward 해줄때 계속 더해주기 때문입니다.
        hypothesis = model(data)
        cost = criterion(hypothesis, target) 
        cost.backward() #backward(): tensor에 대한 자동 미분을 해줍니다.
        optimizer.step() #step(): 옵티마이저에 step()을 불러와 경사하강법을 시작합니다.
        avg_cost += cost / len(train_loader) 
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
 
model.eval()

# 6.
## test 데이터에 적용하는 단계입니다.
test  data에 적용하여 모델의 정확도를 판단합니다.</br>
이때는 torch.no.grad()를 이용하여 gradeient 연산을 잠시 멈추고, </br>
모델에 데이터를 넣어 결과와 실제 값의 결과를 비교하여 정확도를 계산합니다.

In [ ]:
with torch.no_grad(): #torch.no_grad(): gradient 연산을 옵션을 끌 때 사용하는 함수입니다. 이유: gradient는 .grad에 누적으로 저장이 되는데, 이러한 저장을 막기 위해 해당 함수를 사용합니다.
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] 
        total += len(target) 
        correct += (preds==target).sum().item() 
        
    print('Test Accuracy: ', 100.*correct/total, '%')